## Notebook description

+ This notebook uses as an input: 
    + Daily stock prices
    + EPS
    + Earning calls
    + SP500

+ The objective is generate a unique dataframe

## Import libraries

In [3]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os


## Import data

#### Daily stock price

In [4]:
folder_path = '../data/yahoo finance'  # Replace with the actual folder path
all_files = os.listdir(folder_path)
csv_files = [f for f in all_files if f.endswith('.csv')]

dfs = []
for file_name in csv_files:
    df = pd.read_csv(os.path.join(folder_path, file_name))
    df['Ticker'] = file_name.replace('.csv', '')
    dfs.append(df)

stock_p = pd.concat(dfs) 
stock_p = stock_p.reset_index()
stock_p['Date'] = pd.to_datetime(stock_p['Date'],infer_datetime_format=True)

#Save stock prices
stock_p.to_csv('../data/stock_prices.csv')
stock_p

,index,Date,Open,High,Low,Close,Adj Close,Volume,Ticker
0,0,2013-02-01,36.840000,37.200001,36.520000,37.180000,24.526785,9527500,ABBV
1,1,2013-02-04,36.639999,37.299999,36.549999,37.279999,24.592743,6348000,ABBV
2,2,2013-02-05,37.419998,37.730000,37.180000,37.200001,24.539970,7284400,ABBV
3,3,2013-02-06,36.959999,37.279999,36.860001,37.139999,24.500391,5738900,ABBV
4,4,2013-02-07,37.570000,37.570000,35.799999,36.419998,24.025429,9872500,ABBV
...,...,...,...,...,...,...,...,...,...
44091,4608,2023-04-25,157.449997,158.550003,156.309998,157.229996,157.229996,143300,ROG
44092,4609,2023-04-26,156.100006,157.320007,154.110001,154.539993,154.539993,139900,ROG
44093,4610,2023-04-27,155.139999,156.270004,153.449997,155.600006,155.600006,106100,ROG
44094,4611,2023-04-28,167.039993,167.039993,158.570007,160.949997,160.949997,305800,ROG


#### EPS

+ Remitly

In [5]:
#Import base
eps_rem = pd.read_csv('../data/EPS_dataset.csv')

#Change names 
eps_rem['Instrument'] = eps_rem['Instrument'].replace({'ABBV.K': 'ABBV', 'AZN.L': 'AZN','ROG.S': 'ROG'})

#Set to date 
eps_rem['Date'] = pd.to_datetime(eps_rem['Date']).dt.tz_localize(None)

#Rename columns
eps_rem = eps_rem.rename(columns={'Earnings Per Share - Mean': 'eps_mean', 'Earnings Per Share Reported - Mean': 'eps_mean_report','Earnings Per Share - Predicted Surprise PCT': 'eps_mean_report_ps'})

#Drop columns

eps_rem = eps_rem.drop(['Unnamed: 0'],axis=1)
eps_rem

,Instrument,Date,eps_mean,eps_mean_report,eps_mean_report_ps
0,MRK,2005-01-31,2.46963,2.45333,NaN
1,MRK,2005-02-22,2.47259,2.45333,NaN
2,MRK,2005-03-29,2.47185,2.45333,NaN
3,MRK,2005-04-29,2.49333,2.47500,NaN
4,MRK,2005-05-17,2.49423,2.47500,NaN
...,...,...,...,...,...
2155,ROG,2022-08-22,20.94308,18.74167,0.628
2156,ROG,2022-09-29,21.00087,18.87829,-0.217
2157,ROG,2022-10-27,20.77959,18.79200,-0.354
2158,ROG,2022-11-15,20.74096,18.77638,-0.372


+ EPS web: 
    + Source: : https://www.investing.com/equities/abbvie-inc-earnings

In [6]:
eps_web = pd.read_excel('../data/EPS.xlsx')
eps_web['EPS Forecast'] = eps_web['EPS Forecast'].str.replace('/', '')
eps_web['Revenue'] = eps_web['Revenue'].str.replace('B', '')
eps_web['Revenue Forecast'] = eps_web['Revenue Forecast'].str.replace('/', '')

#Changing EPS to numeric

eps_web['EPS'] = pd.to_numeric(eps_web['EPS'], errors='coerce').astype(np.float64)

eps_web['EPS Forecast'] = eps_web['EPS Forecast'].str.strip() # remove leading and trailing spaces
eps_web['EPS Forecast'] = pd.to_numeric(eps_web['EPS Forecast'], errors='coerce').astype(np.float64) # convert to numeric format with 'coerce' option

eps_web

,Release Date,Period End,EPS,EPS Forecast,Revenue,Revenue Forecast,Company
0,2023-07-18,2023-06-01,NaN,1.66,--,13.28B,NVS
1,2023-04-25,2023-03-01,1.71,1.53,12.95,12.75B,NVS
2,2023-02-01,2022-12-01,1.52,1.42,12.69,13.22B,NVS
3,2022-10-25,2022-09-01,1.58,1.55,12.5,12.88B,NVS
4,2022-07-19,2022-06-01,1.56,1.51,12.78,12.88B,NVS
...,...,...,...,...,...,...,...
379,2014-10-31,2014-09-01,0.89,0.77,5.02,"4,832M",ABBV
380,2014-07-25,2014-06-01,0.82,0.76,4.93,"4,701M",ABBV
381,2014-04-25,2014-03-01,0.82,0.68,--,"4,333M",ABBV
382,2014-01-31,2013-12-01,0.82,0.82,--,"5,100M",ABBV


#### SP500

In [7]:
sp_500 = pd.read_csv('../data/sp500data.csv')
sp_500['Date'] =  pd.to_datetime(sp_500['Date'], infer_datetime_format=True)
sp_500['Date'] =  pd.to_datetime(sp_500['Date'], infer_datetime_format=True)
sp_500 = sp_500.rename(columns={'Close_price': 'sp_500_closeprice'})
sp_500

,Date,sp_500_closeprice
0,2006-02-01,128.389999
1,2006-02-02,126.900002
2,2006-02-03,126.269997
3,2006-02-06,126.599998
4,2006-02-07,125.480003
...,...,...
4254,2022-12-23,382.910004
4255,2022-12-27,381.399994
4256,2022-12-28,376.660004
4257,2022-12-29,383.440002


#### Earnings calls reports: 
This is obtained from Financial Modeling Prep API, and has an initial presentation by the CEO(manager), followed by questions by the analyst.

In [8]:
ec10=pickle.load(open("../data/Sentiment_Market_Pharma/earnings_call_top10_ph.pickle", "rb"))
ec10=ec10.reset_index()
ec10=ec10.drop(['index'], axis=1)
ec10['date'] = pd.to_datetime(ec10['date'], infer_datetime_format=True)
ec10

,symbol,quarter,year,date,content
0,PFE,4,2007,2008-01-24,"Operator: Amal Naj, Head of Development. Pleas..."
1,PFE,3,2007,2007-10-18,"Operator: Mr. Amal Naj, Senior VicePresident o..."
2,PFE,2,2007,2007-07-18,"Operator: Amal Naj, Head of Investor Relations..."
3,PFE,4,2008,2009-01-28,Operator: Good morning ladies and gentlemen an...
4,PFE,1,2008,2008-04-17,"Operator: Now we have Amal Naj, Head of Invest..."
...,...,...,...,...,...
510,MRK,4,2020,2021-02-04,"Operator: Good morning. My name is Lara, and I..."
511,MRK,1,2021,2021-04-29,"Operator: Good morning. My name is Lara, and I..."
512,MRK,2,2021,2021-07-29,Operator: Good morning. My name is Mary Serran...
513,MRK,3,2021,2021-10-28,Operator: Good morning. My name is Grace Lakra...


## Generating data frame

In [9]:
merged_df1 = pd.merge(stock_p, eps_rem, left_on=['Date', 'Ticker'], right_on=['Date', 'Instrument'], how = 'left')
merged_df2 = pd.merge(merged_df1, sp_500, left_on=['Date'], right_on=['Date'], how = 'left')
stock_prices = pd.merge(merged_df2, ec10, how='outer', left_on=['Date', 'Ticker'], right_on=['date', 'symbol'])
stock_prices = stock_prices.reset_index(drop=True)
stock_prices=stock_prices.drop(['index'], axis=1)
stock_prices.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,Instrument,eps_mean,eps_mean_report,eps_mean_report_ps,sp_500_closeprice,symbol,quarter,year,date,content
0,2013-02-01,36.840000,37.200001,36.520000,37.180000,24.526785,9527500.0,ABBV,NaN,NaN,NaN,NaN,151.240005,NaN,NaN,NaN,NaT,NaN
1,2013-02-04,36.639999,37.299999,36.549999,37.279999,24.592743,6348000.0,ABBV,NaN,NaN,NaN,NaN,149.539993,NaN,NaN,NaN,NaT,NaN
2,2013-02-05,37.419998,37.730000,37.180000,37.200001,24.539970,7284400.0,ABBV,NaN,NaN,NaN,NaN,151.050003,NaN,NaN,NaN,NaT,NaN
3,2013-02-06,36.959999,37.279999,36.860001,37.139999,24.500391,5738900.0,ABBV,NaN,NaN,NaN,NaN,151.160004,NaN,NaN,NaN,NaT,NaN
4,2013-02-07,37.570000,37.570000,35.799999,36.419998,24.025429,9872500.0,ABBV,ABBV,3.1,2.9,NaN,150.960007,NaN,NaN,NaN,NaT,NaN


In [10]:
# replace NaN values of column of Data with values from column Ticker
stock_prices.loc[stock_prices['Date'].isna(), 'Date'] = stock_prices.loc[stock_prices['Date'].isna(), 'date']
stock_prices.loc[stock_prices['Ticker'].isna(), 'Ticker'] = stock_prices.loc[stock_prices['Ticker'].isna(), 'symbol']

#
stock_prices[['Adj Close', 'Volume']] = stock_prices.groupby('Ticker')[['Adj Close', 'Volume']].fillna(method='ffill')
stock_prices.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,Instrument,eps_mean,eps_mean_report,eps_mean_report_ps,sp_500_closeprice,symbol,quarter,year,date,content
0,2013-02-01,36.840000,37.200001,36.520000,37.180000,24.526785,9527500.0,ABBV,NaN,NaN,NaN,NaN,151.240005,NaN,NaN,NaN,NaT,NaN
1,2013-02-04,36.639999,37.299999,36.549999,37.279999,24.592743,6348000.0,ABBV,NaN,NaN,NaN,NaN,149.539993,NaN,NaN,NaN,NaT,NaN
2,2013-02-05,37.419998,37.730000,37.180000,37.200001,24.539970,7284400.0,ABBV,NaN,NaN,NaN,NaN,151.050003,NaN,NaN,NaN,NaT,NaN
3,2013-02-06,36.959999,37.279999,36.860001,37.139999,24.500391,5738900.0,ABBV,NaN,NaN,NaN,NaN,151.160004,NaN,NaN,NaN,NaT,NaN
4,2013-02-07,37.570000,37.570000,35.799999,36.419998,24.025429,9872500.0,ABBV,ABBV,3.1,2.9,NaN,150.960007,NaN,NaN,NaN,NaT,NaN


In [11]:
stock_prices['Date'] =  pd.to_datetime(stock_prices['Date'], infer_datetime_format=True)
stock_prices = stock_prices.rename(columns={'date': 'earning_call_date'})
stock_prices['earning_call_date'] =  pd.to_datetime(stock_prices['earning_call_date'], infer_datetime_format=True)



In [12]:
# Save dataframe
stock_prices.to_csv('../data/merged_df.csv')